In [61]:
import ccxt
import pandas as pd
import time
from datetime import datetime, timedelta
import numpy as np

In [62]:
colors = ['black', 'blue', 'brown', 'green', 'orange', 'red', 'violet', 'yellow']


# Functions
def fetch_data(exchange, symbol, since, limit=1000):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1h', since=since, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df


def Read_Currency(exchange, symbol_name, time, hours):
    since = exchange.parse8601((datetime.utcnow() - timedelta(hours=hours)).strftime('%Y-%m-%dT%H:%M:%S'))
    df = fetch_data(exchange, symbol_name, since)
    df.set_index('Date', inplace=True)
    dividerTime = str(time) + 'min'
    return df.resample(dividerTime).agg(
        {'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}).ffill()


# find trend output is from 0 to 6.
def FindTrend(data, current_index, previous_index):
    c_array = [data.iloc[current_index].loc['C-Open'], data.iloc[current_index].loc['C-Close'],
               data.iloc[current_index].loc['D-Open'], data.iloc[current_index].loc['D-Close']]
    p_array = [data.iloc[previous_index].loc['C-Open'], data.iloc[previous_index].loc['C-Close'],
               data.iloc[previous_index].loc['D-Open'], data.iloc[previous_index].loc['D-Close']]
    Cmaxt = max(c_array)
    Cmint = min(c_array)
    Cmaxp = max(p_array)
    Cminp = min(p_array)

    Candles_min = min([Cminp, Cmint])
    Candles_max = max([Cmaxp, Cmaxt])

    min_diff = []
    max_diff = []

    a = Cmaxp - Cminp
    b = Cmaxt - Cmint
    if a == 0:
        min_diff.append(1)
        max_diff.append(1)
    else:
        min_diff.append(abs(Cminp - Candles_min) / (Cmaxp - Cminp))
        max_diff.append(abs(Candles_max - Cmaxp) / (Cmaxp - Cminp))

    if b == 0:
        min_diff.append(1)
        max_diff.append(1)
    else:
        min_diff.append(abs(Cmint - Candles_min) / (Cmaxt - Cmint))
        max_diff.append(abs(Candles_max - Cmaxt) / (Cmaxt - Cmint))

    C_meant = (Cmaxt + Cmint) / 2
    C_meanp = (Cmaxp + Cminp) / 2

    if sum(min_diff) <= 0.381 and sum(max_diff) <= 0.381:  # type 3
        return 3
    if sum(min_diff) > 0.381 and sum(max_diff) > 0.381:
        if (Cminp > Cmint) and (Cmaxp < Cmaxt) and \
                (Cminp < C_meant) and (Cmaxp > C_meant):
            return 3
        elif (Cminp < Cmint) and (Cmaxp > Cmaxt) and \
                (Cmint < C_meanp) and (Cmaxt > C_meanp):
            return 3

    if sum(max_diff) <= 0.381:  # type 1 & 4
        if (Cminp < Cmint) and (C_meanp <= Cmaxt) and (C_meant <= Cmaxp):
            return 4
        elif (Cminp > Cmint) and (C_meant <= Cmaxp) and (C_meanp <= Cmaxt):
            return 1

    if sum(min_diff) <= 0.381:  # type 2 & 5
        if (Cmaxp < Cmaxt) and (C_meanp >= Cmint) and (C_meanp >= Cmint):
            return 5
        elif (Cmaxp > Cmaxt) and (C_meant >= Cminp) and (C_meant >= Cminp):
            return 2

    if Cmaxp < Cmaxt and Cminp >= C_meant:
        return 1
    if Cmaxt < Cmaxp and Cmint >= C_meanp:
        return 4
    if Cminp > Cmint and Cmaxp <= C_meant:
        return 5
    if Cmint > Cminp and Cmaxt <= C_meanp:
        return 2
    if C_meanp > C_meant:
        return 0

    if C_meanp < C_meant:
        return 6

    return 7


#---------------------------------------------------------
# trend
# -1: none. 0: downtrend. 1: uptrend
def TrendCheck(data, current_index, previous_index):
    c_array = [data.iloc[current_index].loc['C-Open'], data.iloc[current_index].loc['C-Close'],
               data.iloc[current_index].loc['D-Open'], data.iloc[current_index].loc['D-Close']]
    p_array = [data.iloc[previous_index].loc['C-Open'], data.iloc[previous_index].loc['C-Close'],
               data.iloc[previous_index].loc['D-Open'], data.iloc[previous_index].loc['D-Close']]
    up = 0
    down = 0
    if data.iloc[current_index].loc['T-Mean'] > data.iloc[previous_index].loc['T-Mean']:
        cMax1 = max(c_array)
        cMax2 = max(p_array)
        if cMax1 >= cMax2:
            up = 1
    if data.iloc[current_index].loc['T-Mean'] < data.iloc[previous_index].loc['T-Mean']:
        cMin1 = min(c_array)
        cMin2 = min(p_array)
        if cMin1 <= cMin2:
            down = 1

    if up + down == 2:
        return -1
    elif up == 1:
        return 1
    elif down == 1:
        return 0

    return -1


#--------------------------------------
# candle state from 1 to 6
def CandleState(data, index):
    array1 = [data.iloc[index].loc['C-Open'], data.iloc[index].loc['C-Close'], data.iloc[index].loc['D-Open'],
              data.iloc[index].loc['D-Close']]
    array2 = [data.iloc[index - 1].loc['C-Open'], data.iloc[index - 1].loc['C-Close'],
              data.iloc[index - 1].loc['D-Open'], data.iloc[index - 1].loc['D-Close']]
    array3 = [data.iloc[index - 2].loc['C-Open'], data.iloc[index - 2].loc['C-Close'],
              data.iloc[index - 2].loc['D-Open'], data.iloc[index - 2].loc['D-Close']]
    up = 0
    down = 0
    if max(array1) > max(array2) and \
            max(array1) > max(array3):
        up += 1
    if min(array1) < min(array2) and \
            min(array1) < min(array3):
        down += 1
    if up + down == 2:
        return 1  # both up and down
    elif FindTrend(data, index, index - 1) in [4, 5, 6] and FindTrend(data, index, index - 2) in [5, 6]:
        return 2  # upper
    elif FindTrend(data, index, index - 1) in [0, 1, 2] and FindTrend(data, index, index - 2) in [0, 1]:
        return 3  # lower
    elif data.iloc[index].loc['T-Mean'] >= max(array2):
        return 4  # down-up
    elif data.iloc[index].loc['T-Mean'] <= min(array2):
        return 5  # up-down
    else:
        return 6  # O.W.


#----------------------------------------------------------------------
# Check colors

# get signal color type
def SignalType(index, alpha, beta, data):
    cT = -1
    cP = -1
    for k in range(3, index):
        cT = TrendCheck(data, index, index - k)
        if cT != -1:
            break
    for k in range(3, index - 1):
        cP = TrendCheck(data, index - 1, index - 1 - k)
        if cP != -1:
            break
    if data.iloc[index].loc['vrate'] > alpha:
        if cP == 0 and cT == 0:
            return 'brown'
        elif cP == 1 and cT == 0:
            return 'orange'
        elif cP == 1 and cT == 1:
            return 'blue'
        else:
            return 'violet'
    elif data.iloc[index].loc['vrate'] < beta:
        if cP == 0 and cT == 0:
            return 'black'
        elif cP == 1 and cT == 0:
            return 'yellow'
        elif cP == 1 and cT == 1:
            return 'green'
        else:
            return 'red'

    return 'none'


# Mojesa Func
def Mojesa_Class_Func(ohcl_data):
    new_df = {}

    # date
    new_df['Date'] = ohcl_data.index[0]

    # real date
    new_df['O-Open'] = ohcl_data.iloc[0].loc['Open']
    new_df['O-Close'] = ohcl_data.iloc[-1].loc['Close']
    new_df['O-High'] = max(ohcl_data['High'])
    new_df['O-Low'] = min(ohcl_data['Low'])
    new_df['O-Volume'] = sum(ohcl_data['Volume'])

    if (new_df['O-Open'] >= new_df['O-Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    new_df['O-Bull'] = bullType

    # get order of data candles
    # find orders
    C_Height1 = abs(ohcl_data.iloc[0].loc['Close'] - ohcl_data.iloc[0].loc['Open'])
    C_Height2 = abs(ohcl_data.iloc[1].loc['Close'] - ohcl_data.iloc[1].loc['Open'])
    C_Height3 = abs(ohcl_data.iloc[2].loc['Close'] - ohcl_data.iloc[2].loc['Open'])

    order = np.array(ohcl_data['Volume'].rank(method='max', ascending=False)).flatten()

    if order[0] == order[1] and order[0] == order[2]:
        if C_Height3 < C_Height1 and C_Height3 < C_Height2:
            order[2] = 1
            if C_Height2 < C_Height1:
                order[1] = 2
                order[0] = 3
            else:
                order[1] = 3
                order[0] = 2
        elif C_Height2 < C_Height1:
            order[1] = 1
            if C_Height3 < C_Height1:
                order[2] = 2
                order[0] = 3
            else:
                order[2] = 3
                order[0] = 2
        else:
            order[0] = 1
            if C_Height3 < C_Height2:
                order[2] = 2
                order[1] = 3
            else:
                order[2] = 3
                order[1] = 2
    elif order[0] == order[1]:
        if C_Height2 < C_Height1:
            order[1] -= 1
        else:
            order[0] -= 1
    elif order[0] == order[2]:
        if C_Height3 < C_Height1:
            order[2] -= 1
        else:
            order[0] -= 1
    elif order[1] == order[2]:
        if C_Height3 < C_Height2:
            order[2] -= 1
        else:
            order[1] -= 1

    # ohcl_data['Order'] = order

    #-----------------------------------------------------
    # divide data based on orders
    data1 = []
    data2 = []
    data3 = []

    for i in range(0, ohcl_data.shape[0]):
        match order[i]:
            case 1:
                data1 = ohcl_data.iloc[i].copy()
            case 2:
                data2 = ohcl_data.iloc[i].copy()
            case 3:
                data3 = ohcl_data.iloc[i].copy()

    #-----------------------------------------------------
    # correct devided date time and add bull parameter to them

    Date = ohcl_data.index[0]
    data1.name = Date
    data2.name = Date
    data3.name = Date

    bullType = 'none'
    if (data1['Open'] >= data1['Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    data1['Bull'] = bullType

    bullType = 'none'
    if (data2['Open'] >= data2['Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    data2['Bull'] = bullType

    bullType = 'none'
    if (data3['Open'] >= data3['Close']):
        bullType = 'black'
    else:
        bullType = 'white'
    data3['Bull'] = bullType

    #---------------------------------------------------------
    # calculate combinedData (black/white candle)

    combinedData = data1.copy()

    a = abs(data1['Open'] - data1['Close'])
    b = abs(data2['Open'] - data2['Close'])
    maxindex = pd.Series([a, b]).idxmax()

    openmax = max([data1['Open'], data2['Open']])
    openmin = min([data1['Open'], data2['Open']])

    closemax = max([data1['Close'], data2['Close']])
    closemin = min([data1['Close'], data2['Close']])

    match maxindex:
        case 0:
            if data1['Bull'] == 'black':
                combinedData['Open'] = max([openmax, closemax])
                combinedData['Close'] = min([openmin, closemin])
            else:
                combinedData['Open'] = min([openmin, closemin])
                combinedData['Close'] = max([openmax, closemax])
        case 1:
            if data2['Bull'] == 'black':
                combinedData['Open'] = max([openmax, closemax])
                combinedData['Close'] = min([openmin, closemin])
            else:
                combinedData['Open'] = min([openmin, closemin])
                combinedData['Close'] = max([openmax, closemax])

    combinedData['High'] = max([data1['High'], data2['High']])
    combinedData['Low'] = min([data1['Low'], data2['Low']])
    combinedData['Volume'] = sum([data1['Volume'], data2['Volume']])

    if combinedData['Open'] >= combinedData['Close']:
        combinedData['Bull'] = 'black'
    else:
        combinedData['Bull'] = 'white'

    new_df['C-Open'] = combinedData['Open']
    new_df['C-High'] = combinedData['High']
    new_df['C-Close'] = combinedData['Close']
    new_df['C-Low'] = combinedData['Low']
    new_df['C-Volume'] = combinedData['Volume']
    new_df['C-Bull'] = combinedData['Bull']

    new_df['D-Open'] = data3['Open']
    new_df['D-High'] = data3['High']
    new_df['D-Close'] = data3['Close']
    new_df['D-Low'] = data3['Low']
    new_df['D-Volume'] = data3['Volume']
    new_df['D-Bull'] = data3['Bull']

    new_df['T-Open'] = min([combinedData['Open'], combinedData['Close'], data3['Open'], data3['Close']])
    new_df['T-High'] = new_df['O-High']
    new_df['T-Close'] = max([combinedData['Open'], combinedData['Close'], data3['Open'], data3['Close']])
    new_df['T-Low'] = new_df['O-Low']

    #--------------------------------------------------------------------------
    # calculate data data parameters

    Co_mean = (combinedData['Open'] + combinedData['Close']) / 2.0
    D_mean = (data3['Open'] + data3['Close']) / 2.0
    C_mean = (new_df['T-Open'] + new_df['T-Close']) / 2.0

    body_compare = 0
    x1 = abs(combinedData['Close'] - combinedData['Open'])
    x2 = abs(data3['Close'] - data3['Open'])
    if x2 == x1:
        body_compare = 1
    elif x2 == 0:
        body_compare = 1000
    else:
        body_compare = x1 / x2

    vrate = 0
    if combinedData['Volume'] == 0:
        vrate = 1000
    else:
        vrate = (data3['Volume']) / combinedData['Volume']
    # stRSI , stRSI_K , stRSI_D = StochRSI(combinedData.Close)

    new_df['T-Mean'] = C_mean
    new_df['C-Mean'] = Co_mean
    new_df['D-Mean'] = D_mean
    new_df['brate'] = body_compare
    new_df['vrate'] = vrate

    return new_df


def wait_until_divisible_by_3():
    while True:
        utc_now = datetime.utcnow()
        if utc_now.hour % 3 == 0 and utc_now.minute == 0:
            print(f"Current UTC time {utc_now} is divisible by 3. Starting the process.")
            break
        else:
            print(f"Waiting... Current UTC time: {utc_now}")
        time.sleep(60)


def add_t_s_c(sym_3h, symbol):
    C_Trend = []
    C_State = []
    sigcolor = []
    for i in range(0, sym_3h[symbol].shape[0]):
        if i < 2:
            C_Trend.append({'Date': sym_3h[symbol].index[i], 'Trend': np.nan})
            C_State.append({'Date': sym_3h[symbol].index[i], 'State': np.nan})
            sigcolor.append({'Date': sym_3h[symbol].index[i], 'color': np.nan})
        else:
            C_Trend.append({'Date': sym_3h[symbol].index[i], 'Trend': FindTrend(sym_3h[symbol], i, i - 1)})
            C_State.append({'Date': sym_3h[symbol].index[i], 'State': CandleState(sym_3h[symbol], i)})
            sigcolor.append({'Date': sym_3h[symbol].index[i], 'color': SignalType(i, 0.39, 0.2, sym_3h[symbol])})

    C_Trend = pd.DataFrame(C_Trend)
    C_Trend.set_index('Date', inplace=True)
    C_State = pd.DataFrame(C_State)
    C_State.set_index('Date', inplace=True)
    sigcolor = pd.DataFrame(sigcolor)
    sigcolor.set_index('Date', inplace=True)

    sym_3h[symbol]['Trend'] = C_Trend
    sym_3h[symbol]['State'] = C_State
    sym_3h[symbol]['Color'] = sigcolor

    return sym_3h


def check_conditions(df):
    df = df.reindex()
    for index, row in df.iterrows():
        l = len(df)

        dattee = row['Date']
        color = row['Color']
        state = row['State']
        trend = row['Trend']
        bul = row['O-Bull']

        t_close = row['T-Close']
        t_open = row['T-Open']
        t_low = row['T-Low']
        t_high = row['T-High']

        o_close = row['O-Close']
        o_open = row['O-Open']
        o_low = row['O-Low']
        o_high = row['O-High']

        b_color = 'none'
        a_color = 'none'
        b_index = -1
        a_index = -1

        try:
            for i in range(index - 1, max(index - 5, 0) - 1, -1):
                c_color = df.loc[i, 'Color']
                if c_color in colors:
                    b_color = c_color
                    b_index = i
                    break

            for i in range(index + 1, index + 6):
                c_color = df.loc[i, 'Color']
                if c_color in colors:
                    a_color = c_color
                    a_index = i
                    break
        except:
            continue

        if a_index == -1:
            a_index = min(index + 5, l - 1)
            a_color = 'none'
            a_bul = df.loc[a_index, 'O-Bull']
            a_brate = df.loc[a_index, 'brate']
        else:
            a_bul = df.loc[a_index, 'O-Bull']
            a_brate = df.loc[a_index, 'brate']

        if b_index == -1:
            b_index = max(index - 5, 0)
            b_color = 'none'
            b_bul = df.loc[b_index, 'O-Bull']
            b_brate = df.loc[b_index, 'brate']
        else:
            b_bul = df.loc[b_index, 'O-Bull']
            b_brate = df.loc[b_index, 'brate']

        try:
            next_bul = df.loc[index + 1, 'O-Bull']
        except:
            return ['nothing', None]
        try:
            next_a_bul = df.loc[a_index + 1, 'O-Bull']
        except:
            pass

        if color == 'black' and state == 3 and trend == 0:
            if b_color == 'black' and a_color == 'black':
                #**1
                if bul == 'white':
                    return ['open', [df.index[a_index], 'black.black3.0.black.1', 'long', -1]]
                    # approved_candle = a_index
                    # df.loc[approved_candle, 'signal'] = 'black.black3.0.black.1'
                    # df.loc[approved_candle, 'signal type'] = 'long'
                    # df.loc[approved_candle, 'buy price'] = -1
                    # continue
                #**2
                if bul == 'black' and b_bul == 'white':
                    #*1
                    if a_index != index + 1 and next_bul == 'white':
                        #1
                        if b_brate <= 1 or b_brate >= 4.2:
                            approved_candle = a_index
                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.1'
                            df.loc[approved_candle, 'signal type'] = 'long'
                            df.loc[approved_candle, 'buy price'] = -1
                            continue
                        #2
                        for i in range(index + 1, a_index):
                            candle_bul = df.loc[i, 'bul']
                            if candle_bul == 'black':
                                approved_candle = a_index
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.2'
                                df.loc[approved_candle, 'signal type'] = 'short'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue

                        #3
                        if next_a_bul == 'white':
                            approved_candle = a_index
                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.3'
                            df.loc[approved_candle, 'signal type'] = 'long'
                            df.loc[approved_candle, 'buy price'] = -1
                            continue
                        else:
                            approved_candle = a_index
                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.1.3'
                            df.loc[approved_candle, 'signal type'] = 'short'
                            df.loc[approved_candle, 'buy price'] = -1
                            continue
                    #*2
                    if a_index == index + 1 and next_bul == 'white':
                        #1
                        if b_brate <= 1 or b_brate == 1000:
                            approved_candle = a_index
                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.1'
                            df.loc[approved_candle, 'signal type'] = 'long'
                            df.loc[approved_candle, 'buy price'] = -1
                            continue
                        #2
                        if next_a_bul == 'white':
                            approved_candle = a_index + 1
                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.2'
                            df.loc[approved_candle, 'signal type'] = 'short'
                            df.loc[approved_candle, 'buy price'] = -1
                            continue
                        #3
                        if next_a_bul == 'black':
                            next_next_a_bul = df.loc[a_index + 2, 'bul']
                            if next_next_a_bul == 'black':
                                approved_candle = a_index + 2
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.3'
                                df.loc[approved_candle, 'signal type'] = 'short'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                            else:
                                approved_candle = a_index + 2
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.2.3'
                                df.loc[approved_candle, 'signal type'] = 'long'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                    #*3
                    if next_bul == 'black':
                        flag = 0
                        #1
                        for i in range(index + 1, a_index):
                            candle_bul = df.loc[i, 'bul']
                            if candle_bul == 'white':
                                approved_candle = a_index
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.1'
                                df.loc[approved_candle, 'signal type'] = 'short'
                                df.loc[approved_candle, 'buy price'] = -1
                                flag = 1
                                break
                        if flag == 1:
                            continue
                        #2
                        for i in range(index + 1, a_index):
                            candle_bul = df.loc[i, 'bul']
                            candle_brate = df.loc[i, 'brate']
                            if candle_bul == 'black' and candle_brate <= 1.5:
                                approved_candle = a_index
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.2'
                                df.loc[approved_candle, 'signal type'] = 'long'
                                df.loc[approved_candle, 'buy price'] = -1
                                flag = 1
                            else:
                                approved_candle = a_index
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.2'
                                df.loc[approved_candle, 'signal type'] = 'short'
                                df.loc[approved_candle, 'buy price'] = -1
                                flag = 1

                        if flag == 1:
                            continue

                        #3
                        next_a_cbody_low = min(df.loc[a_index + 1, 'Close'], df.loc[a_index + 1, 'Open'])
                        a_cbody_low = min(df.loc[a_index, 'Close'], df.loc[a_index, 'Open'])
                        if next_a_bul == 'white':
                            next_a_candle_lenght = abs(df.loc[a_index + 1, 't-Close'] - df.loc[a_index + 1, 't-Open'])
                            third_candle_lenght = abs(df.loc[a_index, 't-Close'] - df.loc[a_index, 't-Open'])
                            rate = third_candle_lenght / next_a_candle_lenght
                            if rate >= 3:
                                approved_candle = a_index + 1
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.3'
                                df.loc[approved_candle, 'signal type'] = 'short'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                            elif rate <= 1 and next_a_cbody_low <= a_cbody_low:
                                approved_candle = a_index + 1
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.3'
                                df.loc[approved_candle, 'signal type'] = 'short'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                            else:
                                approved_candle = a_index + 1
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.3'
                                df.loc[approved_candle, 'signal type'] = 'long'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                        #4
                        if next_a_bul == 'black':
                            flag = 0
                            for i in range(a_index + 1, a_index + 6):
                                candle_bul = df.loc[i, 'bul']
                                if candle_bul == 'white':
                                    approved_candle = i
                                    df.loc[approved_candle, 'signal'] = 'black.black3.0.black.2.3.4'
                                    df.loc[approved_candle, 'signal type'] = 'long'
                                    df.loc[approved_candle, 'buy price'] = -1
                                    flag = 1
                                    break
                            if flag == 1:
                                continue
                #**3
                if bul == 'black' and b_bul == 'black':
                    #*1
                    if a_index != index + 1 and next_bul == 'white':
                        #1
                        if a_bul == 'white':
                            #1.1
                            if a_brate <= 1:
                                approved_candle = a_index
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.1_1'
                                df.loc[approved_candle, 'signal type'] = 'long'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                            #1.2
                            next_candle_lenght = abs(df.loc[index + 1, 't-Close'] - df.loc[index + 1, 't-Open'])
                            third_candle_lenght = abs(df.loc[a_index, 't-Close'] - df.loc[a_index, 't-Open'])
                            rate = next_candle_lenght / third_candle_lenght
                            if rate <= 1:
                                approved_candle = a_index
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.1_2'
                                df.loc[approved_candle, 'signal type'] = 'long'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                            else:
                                approved_candle = a_index
                                df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.1_2'
                                df.loc[approved_candle, 'signal type'] = 'short'
                                df.loc[approved_candle, 'buy price'] = -1
                                continue
                        #2
                        if a_bul == 'black':
                            #2.1
                            flag = 0
                            for i in range(index + 1, a_index):
                                candle_bul = df.loc[i, 'bul']
                                candle_brate = df.loc[i, 'brate']
                                if candle_bul == 'white' and candle_brate <= 1:
                                    approved_candle = a_index
                                    df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_1'
                                    df.loc[approved_candle, 'signal type'] = 'short'
                                    df.loc[approved_candle, 'buy price'] = -1
                                    flag = 1
                                    break
                            if flag == 1:
                                continue

                            #2.2
                            # next_a_body_mean = (df.loc[a_index + 1, 't-Close'] + df.loc[a_index + 1, 't-Open']) / 2
                            next_a_body_high = max(df.loc[a_index + 1, 't-Close'], df.loc[a_index + 1, 't-Open'])
                            next_next_a_body_high = max(df.loc[a_index + 2, 't-Close'], df.loc[a_index + 2, 't-Open'])
                            if next_a_bul == 'white':
                                candle_trend = df.loc[a_index + 1, 'trend']
                                if candle_trend <= 2:
                                    approved_candle = a_index + 1
                                    df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                    df.loc[approved_candle, 'signal type'] = 'long'
                                    df.loc[approved_candle, 'buy price'] = -1
                                    continue
                                elif candle_trend == 3:
                                    next_next_a_bul = df.loc[a_index + 2, 'bul']
                                    if next_next_a_bul == 'white' and next_next_a_body_high > next_a_body_high:
                                        approved_candle = a_index + 2
                                        df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                        df.loc[approved_candle, 'signal type'] = 'long'
                                        df.loc[approved_candle, 'buy price'] = -1
                                        continue
                                    else:
                                        approved_candle = a_index + 2
                                        df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                        df.loc[approved_candle, 'signal type'] = 'short'
                                        df.loc[approved_candle, 'buy price'] = -1
                                        continue
                                else:
                                    next_next_a_bul = df.loc[a_index + 2, 'bul']
                                    candle_trend = df.loc[a_index + 2, 'trend']
                                    if next_next_a_bul == 'white' and candle_trend not in [5, 6]:
                                        approved_candle = a_index + 2
                                        df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                        df.loc[approved_candle, 'signal type'] = 'long'
                                        df.loc[approved_candle, 'buy price'] = -1
                                        continue
                                    else:
                                        approved_candle = a_index + 2
                                        df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_2'
                                        df.loc[approved_candle, 'signal type'] = 'short'
                                        df.loc[approved_candle, 'buy price'] = -1
                                        continue

                            #2.3
                            next_a_body_mean = (df.loc[a_index + 1, 't-Close'] + df.loc[a_index + 1, 't-Open']) / 2
                            body_mean = (df.loc[index, 't-Close'] + df.loc[index, 't-Open']) / 2
                            body_low = min(df.loc[index, 't-Close'], df.loc[index, 't-Open'])
                            if next_a_bul == 'black':
                                if body_low < next_a_body_mean < body_mean:
                                    approved_candle = a_index + 1
                                    df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_3'
                                    df.loc[approved_candle, 'signal type'] = 'long'
                                    df.loc[approved_candle, 'buy price'] = -1
                                    continue
                                else:
                                    approved_candle = a_index + 1
                                    df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.1.2_3'
                                    df.loc[approved_candle, 'signal type'] = 'short'
                                    df.loc[approved_candle, 'buy price'] = -1
                                    continue

                    #*2
                    next_a_body_mean = (df.loc[a_index + 1, 't-Close'] + df.loc[a_index + 1, 't-Open']) / 2
                    a_body_high = max(df.loc[a_index, 't-Close'], df.loc[a_index, 't-Open'])
                    next_a_candle_lenght = abs(df.loc[a_index + 1, 't-Close'] - df.loc[a_index + 1, 't-Open'])
                    third_candle_lenght = abs(df.loc[a_index, 't-Close'] - df.loc[a_index, 't-Open'])
                    rate = third_candle_lenght / next_a_candle_lenght
                    if a_index == index + 1 and next_bul == 'white':
                        #1
                        if next_a_bul == 'white':
                            if next_a_body_mean >= a_body_high:
                                if rate <= 1:
                                    approved_candle = a_index + 1
                                    df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.1'
                                    df.loc[approved_candle, 'signal type'] = 'short'
                                    df.loc[approved_candle, 'buy price'] = -1
                                    continue
                                else:
                                    approved_candle = a_index + 1
                                    df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.1'
                                    df.loc[approved_candle, 'signal type'] = 'long'
                                    df.loc[approved_candle, 'buy price'] = -1
                                    continue
                            #2
                            else:
                                flag = 0
                                for i in range(a_index + 2, a_index + 7):
                                    candle_bul = df.loc[i, 'bul']
                                    candle_brate = df.loc[i, 'brate']
                                    if candle_bul == 'black':
                                        if candle_brate > 1 and candle_brate <= 15:
                                            approved_candle = i
                                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.2'
                                            df.loc[approved_candle, 'signal type'] = 'short'
                                            df.loc[approved_candle, 'buy price'] = -1
                                            flag = 1
                                            break
                                        else:
                                            approved_candle = i
                                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.2'
                                            df.loc[approved_candle, 'signal type'] = 'long'
                                            df.loc[approved_candle, 'buy price'] = -1
                                            flag = 1
                                            break
                                if flag == 1:
                                    continue

                        #3
                        if next_a_bul == 'black':
                            approved_candle = a_index + 1
                            df.loc[approved_candle, 'signal'] = 'black.black3.0.black.3.2.3'
                            df.loc[approved_candle, 'signal type'] = 'long'
                            df.loc[approved_candle, 'buy price'] = -1
                            continue

                    #*3
                    #?

        return ['nothing', None]


def find_last_three_colors(df):
    colors_seen = []

    for i in reversed(df.index):
        color = df.at[i, 'color']

        if color == 'none':
            colors_seen.append(color)

        if len(colors_seen) == 3:
            return df.iloc[i:]

    return None

In [59]:
#
exchange = ccxt.mexc()
symbol = 'BTC/USDT'
All_Data = {}
sym_3h = {}

# wait_until_divisible_by_3()

All_Data[symbol] = Read_Currency(exchange, symbol, 1 * 60, 36)
sym_3h[symbol] = []
for i in range(0, 32, 3):
    sym_3h[symbol].append(Mojesa_Class_Func(All_Data[symbol].iloc[i:i + 3]))

sym_3h[symbol] = pd.DataFrame(sym_3h[symbol])
sym_3h[symbol] = add_t_s_c(sym_3h, symbol)[symbol]
sym_3h

{'BTC/USDT':                   Date    O-Open   O-Close    O-High     O-Low     O-Volume  \
 0  2024-08-09 01:00:00  61214.00  61358.40  61563.99  60772.73  1594.770929   
 1  2024-08-09 04:00:00  61358.40  60890.00  61390.70  60588.63  1167.152639   
 2  2024-08-09 07:00:00  60890.00  60759.99  61216.01  60613.84  1226.732733   
 3  2024-08-09 10:00:00  60759.99  60552.00  60844.89  60170.89  1602.456067   
 4  2024-08-09 13:00:00  60552.00  59810.24  61114.70  59535.01  1769.280501   
 5  2024-08-09 16:00:00  59810.24  60314.22  60732.74  59727.40  1481.204967   
 6  2024-08-09 19:00:00  60314.22  60736.00  60979.92  59914.09  1336.659191   
 7  2024-08-09 22:00:00  60736.00  60541.92  61470.58  60292.25  1135.104029   
 8  2024-08-10 01:00:00  60541.92  60437.84  60960.01  60266.03   914.380519   
 9  2024-08-10 04:00:00  60437.84  60458.00  60513.99  60250.00   994.857146   
 10 2024-08-10 07:00:00  60458.00  60896.50  61115.34  60458.00  1342.519828   
 
    O-Bull    C-Open    C-

In [60]:
sym_3h[symbol].columns

Index(['Date', 'O-Open', 'O-Close', 'O-High', 'O-Low', 'O-Volume', 'O-Bull',
       'C-Open', 'C-High', 'C-Close', 'C-Low', 'C-Volume', 'C-Bull', 'D-Open',
       'D-High', 'D-Close', 'D-Low', 'D-Volume', 'D-Bull', 'T-Open', 'T-High',
       'T-Close', 'T-Low', 'T-Mean', 'C-Mean', 'D-Mean', 'brate', 'vrate',
       'Trend', 'State', 'Color'],
      dtype='object')

In [36]:
while True:
    data = find_last_three_colors(sym_3h[symbol])

    check_conditions(data)

    wait_until_divisible_by_3()

    new_data = fetch_data(symbol, since=exchange.parse8601(df['timestamp'].max().strftime('%Y-%m-%dT%H:%M:%S')))

    df = pd.concat([df, new_data]).drop_duplicates().reset_index(drop=True)

    df = df.iloc[3:].reset_index(drop=True)


,Date,O-Open,O-Close,O-High,O-Low,O-Volume,O-Bull,C-Open,C-High,C-Close,...,T-Close,T-Low,T-Mean,C-Mean,D-Mean,brate,vrate,Trend,State,color
0,2024-08-08 06:00:00,56955.62,57230.00,57429.38,56794.36,1010.972480,white,56955.62,57429.38,57216.00,...,57230.00,56794.36,57092.810,57085.810,57209.625,6.389693,0.465782,NaN,NaN,NaN
1,2024-08-08 09:00:00,57230.00,57320.51,57668.00,56986.41,1222.820350,white,57230.00,57668.00,57377.10,...,57377.10,56986.41,57303.550,57303.550,57358.150,3.881266,0.450589,NaN,NaN,NaN
2,2024-08-08 12:00:00,57320.51,58922.25,58992.77,56712.33,1240.625674,white,57927.92,58479.97,57263.53,...,58922.25,56712.33,58092.890,57595.725,58092.890,0.400544,0.411829,5.0,2.0,violet
3,2024-08-08 15:00:00,58922.25,59467.16,59862.36,58648.09,1080.897925,white,59428.02,59862.36,59573.46,...,59573.46,58648.09,59247.855,59500.740,59175.135,0.287562,0.416668,6.0,2.0,violet
4,2024-08-08 18:00:00,59467.16,59506.29,59821.12,59240.58,1600.106695,white,59467.16,59821.12,59328.10,...,59506.29,59240.58,59417.195,59397.630,59454.990,1.355361,0.312238,4.0,2.0,none
5,2024-08-08 21:00:00,59506.29,61688.04,62740.58,59502.55,1203.665603,white,61146.00,62740.58,62331.99,...,62331.99,59502.55,60919.140,61738.995,60326.145,0.723293,0.463036,6.0,2.0,blue
6,2024-08-09 00:00:00,61688.04,61235.50,61741.25,60772.73,1385.705475,black,61214.00,61550.01,61400.00,...,61688.04,60772.73,61451.020,61307.000,61451.020,0.392372,0.221051,4.0,2.0,none
7,2024-08-09 03:00:00,61235.50,60989.00,61563.99,60588.63,1183.321784,black,60720.00,61563.99,61358.40,...,61358.40,60588.63,61039.200,61039.200,61039.200,1.000000,0.400173,0.0,5.0,blue
8,2024-08-09 06:00:00,60989.00,61095.72,61216.01,60613.84,1044.654871,white,60863.57,61110.00,61095.72,...,61095.72,60613.84,60979.645,60979.645,60876.785,8.783579,0.353502,3.0,6.0,none
9,2024-08-09 09:00:00,61095.72,60655.25,61210.31,60170.89,1815.707976,black,61095.72,61210.31,60601.40,...,61095.72,60170.89,60848.560,60848.560,60628.325,9.179573,0.436230,1.0,5.0,orange


In [50]:
(datetime.utcnow() - timedelta(hours=36)).strftime('%Y-%m-%dT%H:%M:%S')

'2024-08-08T20:09:29'